In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rc,  font_manager
import matplotlib
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import statistics
matplotlib.rcParams['axes.unicode_minus'] = False
fontname = "C:/Windows/Fonts/malgun.ttf"
font_name = font_manager.FontProperties(fname=fontname).get_name()
rc('font', family=font_name)

In [2]:
get_data = pd.read_csv('../data/TEMP/경락가격/오이/오이 월별 데이터와 평균편차(전국포함).csv', index_col=None)

In [3]:
get_data.head()

,Date,서울 총거래물량,서울 총거래금액,서울 평균단가,서울 평균단가 편차,부산 총거래물량,부산 총거래금액,부산 평균단가,부산 평균단가 편차,대구 총거래물량,...,대전 총거래물량,대전 총거래금액,대전 평균단가,대전 평균단가 편차,광주 총거래물량,광주 총거래금액,광주 평균단가,광주 평균단가 편차,전국 평균단가,전국 평균단가 편차
0,2020-01-01,2720.0,3808000,1400.000000,75.212591,430514.0,1513741950,3516.127118,-2034.821490,524933.0,...,0.0,0,0.000000,-1493.663393,3760.0,11296800,3004.468085,-1697.063538,3511.307667,-2041.391484
1,2020-02-01,1340.0,3515000,2623.134328,-1147.921737,499632.4,1480696080,2963.570977,-1482.265349,621683.0,...,0.0,0,0.000000,-1087.419308,6250.0,16238900,2598.224000,-1290.819453,2946.967549,-1477.051367
2,2020-03-01,1300.0,1400000,1076.923077,398.289514,1039380.0,1655135842,1592.426102,-111.120474,1336225.0,...,1520.0,2064200,1358.026316,-475.771148,11010.0,21872200,1986.575840,-679.171293,1638.796218,-168.880036
3,2020-04-01,840.0,705500,839.880952,635.331639,1414751.0,1578872900,1116.007623,365.298006,2075460.0,...,975.0,812500,833.333333,207.838209,19990.0,26046300,1302.966483,4.438064,1127.969804,341.946378
4,2020-05-01,4320.0,2747000,635.879630,839.332961,1493296.4,1579825300,1057.944893,423.360735,2256223.0,...,3460.0,3405100,984.132948,529.260378,16270.0,15969726,981.544315,325.860233,1035.129187,434.786995


In [4]:
get_data['전국 총거래물량'] = get_data['서울 총거래물량'] + get_data['부산 총거래물량'] + get_data['대구 총거래물량'] + get_data['대전 총거래물량'] + get_data['광주 총거래물량']
get_data['전국 총거래금액'] = get_data['서울 총거래금액'] + get_data['부산 총거래금액'] + get_data['대구 총거래금액'] + get_data['대전 총거래금액'] + get_data['광주 총거래금액']

In [5]:
seoul_columns = ['Date'] + list(get_data.columns[get_data.columns.str.contains('서울')])
seoul_data = get_data[seoul_columns]

busan_columns = ['Date'] + list(get_data.columns[get_data.columns.str.contains('부산')])
busan_data = get_data[busan_columns]

daegu_columns = ['Date'] + list(get_data.columns[get_data.columns.str.contains('대구')])
daegu_data = get_data[daegu_columns]

daejeon_columns = ['Date'] + list(get_data.columns[get_data.columns.str.contains('대전')])
daejeon_data = get_data[daejeon_columns]

gwangju_columns = ['Date'] + list(get_data.columns[get_data.columns.str.contains('광주')])
gwangju_data = get_data[gwangju_columns]

aways_columns = ['Date'] + list(get_data.columns[get_data.columns.str.contains('전국')])
aways_data = get_data[aways_columns]

In [6]:
aways_data = aways_data[['Date', '전국 총거래물량', '전국 총거래금액', '전국 평균단가', '전국 평균단가 편차']]

In [7]:
seoul_data = seoul_data.rename(columns={'서울 총거래물량':'총거래물량', '서울 총거래금액':'총거래금액', '서울 평균단가':'평균단가', '서울 평균단가 편차':'평균단가 편차'})
busan_data = busan_data.rename(columns={'부산 총거래물량':'총거래물량', '부산 총거래금액':'총거래금액', '부산 평균단가':'평균단가', '부산 평균단가 편차':'평균단가 편차'})
daegu_data = daegu_data.rename(columns={'대구 총거래물량':'총거래물량', '대구 총거래금액':'총거래금액', '대구 평균단가':'평균단가', '대구 평균단가 편차':'평균단가 편차'})
daejeon_data = daejeon_data.rename(columns={'대전 총거래물량':'총거래물량', '대전 총거래금액':'총거래금액', '대전 평균단가':'평균단가', '대전 평균단가 편차':'평균단가 편차'})
gwangju_data = gwangju_data.rename(columns={'광주 총거래물량':'총거래물량', '광주 총거래금액':'총거래금액', '광주 평균단가':'평균단가', '광주 평균단가 편차':'평균단가 편차'})
aways_data = aways_data.rename(columns={'전국 총거래물량':'총거래물량', '전국 총거래금액':'총거래금액', '전국 평균단가':'평균단가', '전국 평균단가 편차':'평균단가 편차'})

In [8]:
seoul_data['지역'] = '서울'
busan_data['지역'] = '부산'
daegu_data['지역'] = '대구'
daejeon_data['지역'] = '대전'
gwangju_data['지역'] = '광주'
aways_data['지역'] = '전국'

In [9]:
all_data = pd.concat([seoul_data, busan_data, daegu_data, daejeon_data, gwangju_data, aways_data])

In [10]:
all_data.to_csv('../data/TEMP/경락가격/오이/오이 컬럼 통합한 데이터와 표준편차.csv', index=False, encoding='UTF-8')